# Programación No Lineal y Dinámica <img src="fcfm.png" style="width: 80px;" style="float: right;"  align="right"  margin-right= "50px"/>

031 E2024 Investigación de Operaciones

Dr. Luis Ángel Gutiérrez Rodríguez

Ismael Sandoval Aguilar 

25 de Febrero de 2024

#### **Programación No Lineal: Situación Problema**

Para ayudar a los propietarios de restaurantes a reabrir sus negocios durante la pandemia de COVID-19, el gobierno creó un índice de riesgo de capacidad generalizado (definido a continuación), o CRI por sus siglas en inglés, para ayudar a los propietarios a comprender cuán riesgoso es abrir sus áreas de asientos interiores y exteriores.

$$CRI = 4x^2_1 - 4x^4_1 + 4x^{6/3}_1 + x_1 x_2 - 4x^2_2 + 4x^4_2$$

Donde:
- $x_1$: Proporción de asientos de interiores disponibles.
- $x_2$: Proporción de asientos de exteriores disponibles.

Mary, una propietaria de restaurante local, quiere reabrir su restaurante con el menor riesgo posible. Sin embargo, comprende que para ofrecer a su personal horas de trabajo valiosas, tendrá que abrir al menos el 10% de su capacidad de asientos interiores y el 25% de su capacidad de asientos exteriores.

¿Cuál es la forma más segura (según el CRI) para que Mary reabra su negocio?

#### Modelación del Problema

Función objetivo:

$$\text{Min} \hspace{2mm} Z = 4x^2_1 - 4x^4_1 + 4x^{6/3}_1 + x_1 x_2 - 4x^2_2 + 4x^4_2$$

Sujeto a las restricciones:

- $x_1 \geq 0.1$
- $x_2 = 0.25$
- $x_1, x_2 \leq 1$

#### Modelación Computacional

In [4]:
from pyomo.environ import *

In [2]:
# Crear un modelo
model = ConcreteModel()

# Declarar variables
model.x1 = Var(bounds=(0.1, 1))
model.x2 = Var(bounds=(0.25, 1))

# Definir la función objetivo
model.CRI = Objective(expr=4*model.x1**2 - 4*model.x1**4 + 4*model.x1**(6/3) + model.x1*model.x2 - 4*model.x2**2 + 4*model.x2**4, sense=minimize)

# Resolver el modelo utilizando MindtPy
solver = SolverFactory('mindtpy')
solution = solver.solve(model, mip_solver='glpk', nlp_solver='ipopt', strategy='OA')

In [3]:
# Mostrar resultados
optimal_x1 = model.x1.value  # Proporción óptima de asientos interiores
optimal_x2 = model.x2.value  # Proporción óptima de asientos exteriores
optimal_CRI = value(model.CRI)  # Valor óptimo del CRI

print(f"Proporción óptima de asientos interiores (x1): {optimal_x1}")
print(f"Proporción óptima de asientos exteriores (x2): {optimal_x2}")
print(f"Índice de riesgo óptimo (CRI): {optimal_CRI}")

Proporción óptima de asientos interiores (x1): 0.0999999911857276
Proporción óptima de asientos exteriores (x2): 0.7007719058283692
Índice de riesgo óptimo (CRI): -0.8500046541482943


#### **Programación Dinámica: Situación Problema**

El problema de cambio de monedas aborda la forma de encontrar el número mínimo de monedas (de ciertas denominaciones) tales que entre ellas suman una cierta cantidad. Es un tipo de problema de la mochila, y tiene aplicaciones que exceden el ámbito específico de la circulación de dinero.

#### Modelación del Problema

Conjuntos y parámetros:

- $C$: Conjunto de denominaciones de monedas disponibles, donde $c \in C$ representa cada denominación.
- $A$: Cantidad total de dinero a formar usando las monedas.
- $v_c$: Valor de la moneda de denominación $c$.

Variables de Decisión:

- $x_c$: Número de monedas de denominación $c$ a utilizar, $x_c$ es un entero no negativo para todo $c \in C$.

Función objetivo:

$$ \text{Minimizar} \; Z = \sum_{c \in C} x_c $$

Sujeto a:
- Restricción de cantidad: Asegurar que el valor total de las monedas utilizadas iguale a la cantidad objetivo $A$:
$$ \sum_{c \in C} v_c \cdot x_c = A $$

- No negatividad e integridad: Cada $x_c$ debe ser un entero no negativo, reflejando el conteo de monedas físicas:
$$ x_c \geq 0 \; \text{y entero para todo} \; c \in C $$

#### Modelación Computacional

In [2]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, NonNegativeIntegers, minimize

In [3]:
# Datos del problema
monedas = [1, 2, 5]  # Denominaciones de monedas disponibles
cantidad = 11  # Cantidad objetivo a alcanzar

# Crear un modelo concreto de Pyomo
modelo = ConcreteModel()

# Variables de decisión
modelo.x = Var(monedas, domain=NonNegativeIntegers)

# Función objetivo: Minimizar el número total de monedas
modelo.objetivo = Objective(expr=sum(modelo.x[c] for c in monedas), sense=minimize)

# Restricción: La suma de los valores de las monedas debe ser igual a la cantidad deseada
modelo.restriccion_cantidad = Constraint(expr=sum(c * modelo.x[c] for c in monedas) == cantidad)

# Resolver el modelo usando el solucionador GLPK
solucionador = SolverFactory('glpk')
resultado = solucionador.solve(modelo)

In [1]:
# Verificar y mostrar la solución
if resultado.solver.status == 'ok' and resultado.solver.termination_condition == 'optimal':
    print("Solución óptima encontrada:")
    for moneda in monedas:
        print(f"Monedas de {moneda}: {modelo.x[moneda].value}")
else:
    print("No se encontró solución óptima.")


Solución óptima encontrada:
Monedas de 1: 1.0
Monedas de 2: 0.0
Monedas de 5: 2.0
